<a href="https://colab.research.google.com/github/alexlopespereira/minicursoIDP2021/blob/master/ETL_Vacinacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primeiros passos com o BigQuery no Google Colab
*   Baseado no notebook https://colab.research.google.com/notebooks/bigquery.ipynb

*   Execute os passos abaixo, conforme explicado nos slides:
1.   Use o [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) para **criar um projeto na Google Cloud Platform (GCP)**, se você ainda não tem um.
2.   [Habilite as APIs do BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) para o projeto.

* Ou consulte o [Tutorial em vídeo](https://www.youtube.com/watch?v=JLXLCv5nUCE) 

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [80]:
import pandas as pd
## Defina o id do seu projeto no bigquery!!!!!
project_id = 'mscovid' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!

df_vacina = pd.io.gbq.read_gbq('''
  SELECT * FROM
(
  -- #1 from_item
  SELECT mv.sigla_uf,  mv.nome_fabricante,  mv.vacina, me.id_municipio, REGEXP_REPLACE(mv.dose, '[^1-4A-Za-z]','')  as dose,
  EXTRACT (MONTH FROM mv.data_aplicacao) AS mes,
  EXTRACT(YEAR FROM mv.data_aplicacao) AS ano,
  FROM basedosdados.br_ms_vacinacao_covid19.microdados_vacinacao mv
  JOIN basedosdados.br_ms_vacinacao_covid19.microdados_estabelecimento me ON me.id_estabelecimento = mv.id_estabelecimento
  GROUP BY
    mv.sigla_uf, mv.nome_fabricante, mv.vacina, me.id_municipio, mv.dose, mes, ano
  LIMIT 50000000
)
PIVOT
(
  -- #2 aggregate
  COUNT(1) AS QTD
  -- #3 pivot_column
  FOR dose in ('1Dose', '2Dose', '3Dose', 'Dose', 'nica', '1DoseRevacinao', '2DoseRevacinao', 'DoseInicial', 'DoseAdicional', '1Reforo')
)
''', project_id=project_id)

df_vacina.head()

,sigla_uf,nome_fabricante,vacina,id_municipio,mes,ano,QTD_1Dose,QTD_2Dose,QTD_3Dose,QTD_Dose,QTD_nica,QTD_1DoseRevacinao,QTD_2DoseRevacinao,QTD_DoseInicial,QTD_DoseAdicional,QTD_1Reforo
0,MA,PFIZER,87,2111300,8,2021,1,1,0,0,0,0,0,0,1,0
1,MA,ASTRAZENECA/FIOCRUZ,85,2112902,7,2021,1,1,0,0,0,0,0,0,0,0
2,MA,SINOVAC/BUTANTAN,86,2106607,9,2021,1,1,0,0,0,0,0,0,0,0
3,MA,ASTRAZENECA/FIOCRUZ,85,2112506,9,2021,1,1,0,0,0,0,0,0,0,0
4,MA,SINOVAC/BUTANTAN,86,2106607,7,2021,1,1,0,0,0,0,0,0,0,0


##Próximo passo: agregar a posição geográfica do centroid dos municípios e da sua população

## Pré-processamento dos centroids dos municipios

In [4]:
!pip install dbf

     |████████████████████████████████| 107 kB 4.7 MB/s 
     |████████████████████████████████| 123 kB 29.4 MB/s 


In [6]:
#Fonte oficial https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/27385-localidades.html?=&t=downloads

!wget https://github.com/renatocol/Latitude_Longitude_Brasil/raw/master/BR_Localidades_2010.dbf

--2021-10-28 20:03:58--  https://github.com/renatocol/Latitude_Longitude_Brasil/raw/master/BR_Localidades_2010.dbf
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/renatocol/Latitude_Longitude_Brasil/master/BR_Localidades_2010.dbf [following]
--2021-10-28 20:03:58--  https://raw.githubusercontent.com/renatocol/Latitude_Longitude_Brasil/master/BR_Localidades_2010.dbf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17794056 (17M) [application/octet-stream]
Saving to: ‘BR_Localidades_2010.dbf’

BR_Localidades_2010 100%[===================>]  16.97M   110MB/s    in 0.2s    

2021-10-28 20:04:00 (110 

In [7]:
import dbf
import pandas as pd

table = dbf.Table(filename='./BR_Localidades_2010.dbf')
table.open(dbf.READ_ONLY)
df = pd.DataFrame(table)
table.close()

print(df)

          0                     1           2   ...         19          20   21
0          1  110001505000001       URBANO      ... -11.935540  337.735719  0.0
1          2  110001515000001       URBANO      ... -12.437239  215.244429  0.0
2          3  110001520000001       URBANO      ... -12.601415  181.044807  0.0
3          4  110001525000001       URBANO      ... -11.919792  191.576571  0.0
4          5  110001530000001       URBANO      ... -13.079806  157.285277  0.0
...      ...                   ...         ...  ...        ...         ...  ...
21881  21882  530010805180237       URBANO      ... -15.939671  911.712363  0.0
21882  21883  530010805180238       URBANO      ... -15.936009  926.632968  0.0
21883  21884  530010805180314       URBANO      ... -15.939968  902.635257  0.0
21884  21885  530010805200120       URBANO      ... -15.939726  921.346973  0.0
21885  21886  530010805200123       URBANO      ... -15.947606  953.389949  0.0

[21886 rows x 22 columns]


In [14]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,1,110001505000001,URBANO,110001505006,Redondo ...,11000150500,...,110001505,ALTA FLORESTA D'OESTE ...,1100015,ALTA FLORESTA D'OESTE ...,CACOAL ...,LESTE RONDONIENSE ...,RONDÔNIA ...,1,05,CIDADE ...,ALTA FLORESTA D'OESTE ...,-61.999824,-11.935540,337.735719,0.0
1,2,110001515000001,URBANO,,...,11000151500,...,110001515,FILADÉLFIA D'OESTE ...,1100015,ALTA FLORESTA D'OESTE ...,CACOAL ...,LESTE RONDONIENSE ...,RONDÔNIA ...,2,15,VILA ...,FILADÉLFIA D'OESTE ...,-62.043898,-12.437239,215.244429,0.0
2,3,110001520000001,URBANO,,...,11000152000,...,110001520,IZIDOLÂNDIA ...,1100015,ALTA FLORESTA D'OESTE ...,CACOAL ...,LESTE RONDONIENSE ...,RONDÔNIA ...,2,20,VILA ...,IZIDOLÂNDIA ...,-62.175549,-12.601415,181.044807,0.0
3,4,110001525000001,URBANO,,...,11000152500,...,110001525,NOVA GEASE D'OESTE ...,1100015,ALTA FLORESTA D'OESTE ...,CACOAL ...,LESTE RONDONIENSE ...,RONDÔNIA ...,2,25,VILA ...,NOVA GEASE D'OESTE ...,-62.318650,-11.919792,191.576571,0.0
4,5,110001530000001,URBANO,,...,11000153000,...,110001530,ROLIM DE MOURA DO GUAPORÉ ...,1100015,ALTA FLORESTA D'OESTE ...,CACOAL ...,LESTE RONDONIENSE ...,RONDÔNIA ...,2,30,VILA ...,ROLIM DE MOURA DO GUAPORÉ ...,-62.276812,-13.079806,157.285277,0.0


In [55]:
# Algebra relacional: Projeção. Ou seja, selecionar colunas
# Equivalente SQL: select col2, col9, col18, col19 from df
df_geo = df[[2,9,18,19]].rename(columns={2:"tipo_localidade",9:"cod_ibge",18:"long",19:"lat"})
df_geo['cod_ibge'] = df_geo['cod_ibge'].str.strip()
df_geo['tipo_localidade'] = df_geo['tipo_localidade'].str.strip()
# Algebra relacional: Selecionar/Filtrar/Restringir. Ou seja, filtrar linhas da tabela
# Equivalente SQL: select * from df where tipo_localidade='URBANO'
df_geo = df_geo[df_geo['tipo_localidade']=='URBANO']
# Equivalente SQL: select * from df where cod_ibge!=''
df_geo = df_geo[df_geo['cod_ibge']!='']

# Algebra relacional: Agregação. Ou seja, agrupar registros e extrair alguma métrica de cada conjunto da agregação.
df_geo_agg = df_geo.groupby('cod_ibge').first().reset_index()
df_geo_agg.head()

,cod_ibge,tipo_localidade,long,lat
0,1100015,URBANO,-61.999824,-11.935540
1,1100023,URBANO,-63.033269,-9.908463
2,1100031,URBANO,-60.544314,-13.499763
3,1100049,URBANO,-61.442944,-11.433865
4,1100056,URBANO,-60.818426,-13.195033


In [56]:
df_geo_agg['lat_long'] = df_mun_agg[['lat','long']].apply(lambda x: f"{str(x['lat']).replace(',','.')},{str(x['long']).replace(',','.')}", axis=1)
df_geo_agg.head()
# df_recorte_mun.head()
# df.loc[0,2]

,cod_ibge,tipo_localidade,long,lat,lat_long
0,1100015,URBANO,-61.999824,-11.935540,"-11.9355403048,-61.9998238963"
1,1100023,URBANO,-63.033269,-9.908463,"-9.90846286657,-63.033269278"
2,1100031,URBANO,-60.544314,-13.499763,"-13.4997634597,-60.5443135812"
3,1100049,URBANO,-61.442944,-11.433865,"-11.4338650287,-61.4429442118"
4,1100056,URBANO,-60.818426,-13.195033,"-13.195033032,-60.8184261647"


## Pré-processamento da população dos municípios

In [20]:
dfpop = pd.read_excel("https://github.com/alexlopespereira/minicursoIDP2021/raw/master/data/estimativa_dou_2021.xls", skiprows=1, sheet_name="Municípios", dtype={"COD. UF": "str", "COD. MUNIC": "str"})
dfpop.head()

,UF,COD. UF,COD. MUNIC,NOME DO MUNICÍPIO,POPULAÇÃO ESTIMADA
0,RO,11,00015,Alta Floresta D'Oeste,22516
1,RO,11,00023,Ariquemes,111148
2,RO,11,00031,Cabixi,5067
3,RO,11,00049,Cacoal,86416
4,RO,11,00056,Cerejeiras,16088


In [64]:
dfpop['cod_ibge'] = dfpop[['COD. UF', 'COD. MUNIC']].apply(lambda x: f"{x['COD. UF']}{str(x['COD. MUNIC'])}", axis=1)
dfpop.rename(columns={"NOME DO MUNICÍPIO":"nome_municipio", "POPULAÇÃO ESTIMADA": "populacao"}, inplace=True)
dfpop.head()


,UF,COD. UF,COD. MUNIC,nome_municipio,populacao,cod_ibge
0,RO,11,00015,Alta Floresta D'Oeste,22516,1100015
1,RO,11,00023,Ariquemes,111148,1100023
2,RO,11,00031,Cabixi,5067,1100031
3,RO,11,00049,Cacoal,86416,1100049
4,RO,11,00056,Cerejeiras,16088,1100056


## Join/Merge das tres tabelas

In [81]:
#Algebra relacional: Join. Ou seja, junção de tabelas.
dfvacina_etl = df_vacina.merge(df_geo_agg[['cod_ibge','lat_long']], how='left', left_on='id_municipio', right_on='cod_ibge') \
                        .merge(dfpop[['nome_municipio','populacao','cod_ibge']], how='left', left_on='id_municipio', right_on='cod_ibge')
dfvacina_etl.head()

,sigla_uf,nome_fabricante,vacina,id_municipio,mes,ano,QTD_1Dose,QTD_2Dose,QTD_3Dose,QTD_Dose,QTD_nica,QTD_1DoseRevacinao,QTD_2DoseRevacinao,QTD_DoseInicial,QTD_DoseAdicional,QTD_1Reforo,cod_ibge_x,lat_long,nome_municipio,populacao,cod_ibge_y
0,MA,PFIZER,87,2111300,8,2021,1,1,0,0,0,0,0,0,1,0,2111300,"-2.53188598508,-44.2979185642",São Luís,1115932,2111300
1,MA,ASTRAZENECA/FIOCRUZ,85,2112902,7,2021,1,1,0,0,0,0,0,0,0,0,2112902,"-3.48204736455,-44.8695722406",Vitória do Mearim,32956,2112902
2,MA,SINOVAC/BUTANTAN,86,2106607,9,2021,1,1,0,0,0,0,0,0,0,0,2106607,"-5.52246501943,-43.2012082829",Matões,34099,2106607
3,MA,ASTRAZENECA/FIOCRUZ,85,2112506,9,2021,1,1,0,0,0,0,0,0,0,0,2112506,"-2.76355019689,-42.2768472077",Tutóia,59927,2112506
4,MA,SINOVAC/BUTANTAN,86,2106607,7,2021,1,1,0,0,0,0,0,0,0,0,2106607,"-5.52246501943,-43.2012082829",Matões,34099,2106607


In [82]:
dfvacina_etl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194894 entries, 0 to 194893
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   sigla_uf            194894 non-null  object
 1   nome_fabricante     194844 non-null  object
 2   vacina              194894 non-null  object
 3   id_municipio        194894 non-null  object
 4   mes                 194894 non-null  int64 
 5   ano                 194894 non-null  int64 
 6   QTD_1Dose           194894 non-null  int64 
 7   QTD_2Dose           194894 non-null  int64 
 8   QTD_3Dose           194894 non-null  int64 
 9   QTD_Dose            194894 non-null  int64 
 10  QTD_nica            194894 non-null  int64 
 11  QTD_1DoseRevacinao  194894 non-null  int64 
 12  QTD_2DoseRevacinao  194894 non-null  int64 
 13  QTD_DoseInicial     194894 non-null  int64 
 14  QTD_DoseAdicional   194894 non-null  int64 
 15  QTD_1Reforo         194894 non-null  int64 
 16  co

In [70]:
dfvacina_etl.to_gbq("testes_idp.vacinacao",
              project_id=project_id,
              chunksize=40000,
              if_exists='replace',
              )

1it [00:03,  3.91s/it]


In [79]:
dfvacina_etl.to_gbq("testes_idp.vacinacao_5000000",
              project_id=project_id,
              chunksize=40000,
              if_exists='replace',
              )

5it [00:32,  6.58s/it]
